In [ ]:
import optuna
import pandas as pd
import numpy as np
import yaml
import pickle

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import tensor, cat, save, load, optim, nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from models.resnet18_model import ResNet18

In [ ]:
# Optuna objective
def Objective(trial):
    x = trial.suggest_float("x", -10, 10)
    return x**2/(10-x)

In [ ]:
# Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(Objective, n_trials=50)

In [ ]:
print("Best value:", study.best_value)
print("Best params:", study.best_params)

In [ ]:
#-------------------#
#  Prepare dataset  #
#-------------------#
from dataset_handler import processDataset, loadDataset
import study_handler as sh

# Dataset config
data_cfg = { 
            "dataset": "cifar10",
            "f_train": 0.5,
            "f_test": 0.5,
            "augment": False,
            "root": "./data"
            }

study_cfg = {
    "trials": 10
}

trainset, testset = loadDataset(data_cfg)
train_dataset, test_dataset = processDataset(data_cfg, trainset, testset)

# Assign the datasets
sh.train_dataset = train_dataset
sh.test_dataset = test_dataset

In [ ]:
# ------------------------#
#        Run study        #
# ------------------------#
study_cfg = {
    "study_name": "cifar10_without_augments",
    "trials": 10
}

def run_optimization():
    storage = JournalStorage(JournalFileBackend(file_path="./journal.log"))
    study = optuna.create_study(
        study_name=study_cfg["study_name"],
        storage=storage,
        load_if_exists=True,
        direction="maximize"
    )
    study.optimize(sh.objective, n_trials=study_cfg["trials"])
    print(f"Study '{study_cfg["study_name"]}' completed. Best value: {study.best_value}, params: {study.best_params}")

run_optimization()

In [ ]:
# Save training and config metadata. Folder name index-hashed_config/logits, metadata, etc
form save-load import save, hashCfg


In [ ]:
# Train shadow models